# II. Plot number of bees scenting away from queen (0,0)

Goal: Plot proportion of correctly scenting bees / total scenting bees.

In [257]:
# Imports
import json
import numpy as np
import plotly.plotly as py
import matplotlib as mpl
from matplotlib import cycler
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import os
import glob2
import re
import random
from sklearn.preprocessing import normalize
import math


In [242]:
# Use the data generated in position_data
with open("position_data/Q0.15_W0.005_D0.55_T0.005_wb1.json", "r") as f:
    position_data = json.load(f)

In [243]:
# Replicate, bee, time step
len(position_data), len(position_data[0]), len(position_data[0]["worker_1"])

(10, 50, 320)

Look at all the scenting timesteps of a single worker. It stays still for those 6 timesteps, so all same position and bias.

In [258]:
def scenting_of_1_bee_all_t(position_data, replicate_i, worker_i):
    ''' Look at one replicate: a bee's scenting, position, and bias info over 
    all time steps and determine if bee is scenting correctly away from queen 
    at 0,0. Return a list of length of all time steps (e.g. 320) showing T or 
    F for that bee.
    '''
    
#     with open("position_data/{}".format(json_name), "r") as f:
#         position_data = json.load(f)
    
    correct_scenting_1_bee_all_t = []
    CORRECT_SCENTING_THRESHOLD = -0.5

    # k = 0
    for t_i, t_info in enumerate(position_data[replicate_i]["worker_{}".format(worker_i)]):
        scenting = position_data[replicate_i]["worker_{}".format(worker_i)][t_i]["scenting"]

    #     if k > 100:
    #         break

        if scenting == True: 
            bias = (position_data[replicate_i]["worker_{}".format(worker_i)][t_i]["bias_x"], position_data[replicate_i]["worker_{}".format(worker_i)][t_i]["bias_y"])
            # print("bias_xy: {}".format(bias))
            position = (position_data[replicate_i]["worker_{}".format(worker_i)][t_i]["x"], position_data[replicate_i]["worker_{}".format(worker_i)][t_i]["y"])
            # print("position: {}".format(position))

            # Determine if scenting in queen's direction
            r = -np.array(position)         # Negative of vector Worker to Queen
            u_r = r / np.linalg.norm(r)     # Unit vector 

            # Get dot product of direction W->Q unit vector and the unit vector of scenting direction
            similarity = np.dot(u_r, bias)  # bound between -1 and 1
            # print("Similarity: {} \n".format(similarity))

            # If within threshold allowed, bee is scenting correctly
            correct_scenting = True if similarity <= CORRECT_SCENTING_THRESHOLD else False

            # If correct_scenting, add True to correct_scenting_1_bee_all_t, else False
            correct_scenting_1_bee_all_t.append(True) if correct_scenting == True else correct_scenting_1_bee_all_t.append(False)

    #         k += 1

    #         if k % 1 == 0:

    #             plt.title(similarity)
    #             plt.scatter([0], [0], c='k')
    #             plt.scatter(*position, c='r')
    #             plt.quiver(*position, *bias)
    #             plt.quiver(*position, *u_r)
    #             plt.quiver(*position, *-u_r)
    #             plt.xlim([-0.5, 0.5])
    #             plt.ylim([-1, 3]) 
    #             plt.figure()

        else: 
            correct_scenting_1_bee_all_t.append(False)
            
    return correct_scenting_1_bee_all_t

In [261]:
# Test function
json_name = "Q0.15_W0.005_D0.55_T0.5_wb1.json"
with open("position_data/{}".format(json_name), "r") as f:
    position_data = json.load(f)
        
one_bee_over_t = scenting_of_1_bee_all_t(position_data, replicate_i=0, worker_i=9)
# one_bee_over_t

Get correct_scenting_1_bee_all_t lists for all 50 workers in one replicate json. Then average over all 10 replicates to get one cumulative list.

In [296]:
def scenting_of_all_bees_all_t(position_data, rep_i):
    ''' Correct scenting info for all bees over all time for a single replicate. 
    Get list of length 50, each list inside is length 320 (all t). 
    Count number of scenting bees at each timestep. Return this list. '''

    all_bees_all_t = []

    for bee_i, bee_info in enumerate(position_data[rep_i]):
        one_bee = scenting_of_1_bee_all_t(position_data, replicate_i=rep_i, worker_i=bee_i+1)
        all_bees_all_t.append(one_bee)
        
    # Count number of correctly scenting bees over time 
    # Combine all separate lists for bees into this list; each list inside is a timestep over all bees
    ### Get scenting for all bees over all time
    over_all_time = []

    # Loop over all timesteps
    for time_i in range(len(all_bees_all_t[0])):
        # print(time_i)
        # Loop over each of 50 lists in all_bees_all_t  
        one_t = []
        for bee_i in range(len(all_bees_all_t)):
            scenting = all_bees_all_t[bee_i][time_i]
            one_t.append(scenting)
        over_all_time.append(one_t)
        
    ### Count scenting bees in swarm over time
    swarm_over_time = []
    for t in over_all_time:
        scenting_true = sum(t)
        swarm_over_time.append(scenting_true)
    
    
    return swarm_over_time

In [299]:
all_bees_over_t = scenting_of_all_bees_all_t(position_data, 1)
all_bees_over_t

[0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 3,
 2,
 2,
 2,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 3,
 3,
 3,
 4,
 5,
 5,
 4,
 4,
 4,
 4,
 3,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 3,
 3,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 3,
 5,
 5,
 6,
 6,
 6,
 5,
 4,
 3,
 3,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 3,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 3,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 3,
 3,
 4,
 3,
 3,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:
def scenting_of_all_replicates(position_data):
    ''' Correct scenting info for all replicates. '''
    